Chrome Driver 다운로드 URL : http://chromedriver.chromium.org/downloads


1-3. 라이브러리 Import 및 Chrome Driver 실행

In [1]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [2]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.

# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
# driver = webdriver.Chrome(executable_path = "chromedriver", chrome_options=chrome_options)

# MAC, Linux 운영체제
driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)

<ipython-input-2-d1a0fa76bc01>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)
<ipython-input-2-d1a0fa76bc01>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)


2. 뉴스기사 및 댓글 수집하기


2-1. 뉴스기사 정보 입력

### 불필요 표시기호 제거

In [3]:
import re

class ArticleParser(object):
    special_symbol = re.compile('[\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$&▲▶◆◀■【】\\\=\(\'\"]')
    content_pattern = re.compile('본문 내용|TV플레이어| 동영상 뉴스|flash 오류를 우회하기 위한 함수 추가function  flash removeCallback|tt|앵커 멘트|xa0')

    @classmethod
    def clear_content(cls, text):
        # 기사 본문에서 필요없는 특수문자 및 본문 양식 등을 다 지움
        newline_symbol_removed_text = text.replace('\\n', '').replace('\\t', '').replace('\\r', '')
        special_symbol_removed_content = re.sub(cls.special_symbol, ' ', newline_symbol_removed_text)
        end_phrase_removed_content = re.sub(cls.content_pattern, '', special_symbol_removed_content)
        blank_removed_content = re.sub(' +', ' ', end_phrase_removed_content).lstrip()  # 공백 에러 삭제
        reversed_content = ''.join(reversed(blank_removed_content))  # 기사 내용을 reverse 한다.
        content = ''
        for i in range(0, len(blank_removed_content)):
            # reverse 된 기사 내용중, ".다"로 끝나는 경우 기사 내용이 끝난 것이기 때문에 기사 내용이 끝난 후의 광고, 기자 등의 정보는 다 지움
            if reversed_content[i:i + 2] == '.다':
                content = ''.join(reversed(reversed_content[i:]))
                break
        return content

    @classmethod
    def clear_headline(cls, text):
        # 기사 제목에서 필요없는 특수문자들을 지움
        newline_symbol_removed_text = text.replace('\\n', '').replace('\\t', '').replace('\\r', '')
        special_symbol_removed_headline = re.sub(cls.special_symbol, '', newline_symbol_removed_text)
        return special_symbol_removed_headline

2-2. 뉴스기사 수집

#### 뉴스 검색 날짜
1.2020 11 01 ~ 2021 02 01

2.2021 02 01 ~ 2021 05 01

3.2021 05 01 ~ 2021 08 01

4.2021 08 01 ~ 2021 11 01

In [4]:
import time
from datetime import date, timedelta

In [5]:
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + timedelta(i)

In [6]:
start_date = date(2021, 9, 2) #변경할 부분
end_date = date(2021, 10, 1) #변경할 부분

for single_date in daterange(start_date, end_date):
    search_date = single_date.strftime("%Y.%m.%d")

    
     # 수집할 뉴스기사 정보를 입력합니다.
    QUERY = "주식"           # 필터링 키워드
    START_DATE = search_date # 필터링 일자 (작성일 기준)
    END_DATE = search_date
    START_PAGE = 1            # 검색결과 저장 페이지 범위 (네이버 뉴스기사는 검색결과 중 최대 4,000페이지만 제공)
    END_PAGE = 50             #변경할 부분
                              # 기사와 댓글을 저장할 파일명
    article_filename = "article_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"
    comment_filename = "comment_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"
    

    fa = open(article_filename, "w", encoding="utf-8")

    news_count = 0

    for page in range(START_PAGE, END_PAGE*10+1, 10):
        print(page,  "번째 기사부터 최대 10개 수집중...", end="\r")
        URL = "https://search.naver.com/search.naver?&where=news&query=" + QUERY 
        URL += "&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds="
        URL += START_DATE + "&de=" + END_DATE + "&docid=&&start=" + str(page) + "&refresh_start=0"
        driver.get(URL)
        time.sleep(2)

        try:
            news_list = driver.find_element(By.CLASS_NAME, "list_news").find_elements(By.CLASS_NAME, "bx")
        except:
            break


        news_count += len(news_list)
        for news in news_list[:]:
            link_list = news.find_element(By.CLASS_NAME, "info_group").find_elements(By.TAG_NAME, "a")
            if len(link_list) == 1: ### ? 페이지수가 하나면
                continue
            article_url = link_list[1].get_attribute("href").strip()
            link_list[1].click()    ### ? 첫번째 기사 클릭
            time.sleep(1)
            current_window = driver.current_window_handle

            try:
                new_window = [window for window in driver.window_handles if window != current_window][0]
                driver.switch_to.window(new_window)
            except:
                driver.switch_to.window(current_window)
                continue
            time.sleep(1)

            try:
                try:
                    source_label = driver.find_element(By.CLASS_NAME, "press_logo") ###언론사명 저장
                except:
                    source_label = driver.find_element(By.ID, "pressLogo")
                source_img = source_label.find_element(By.TAG_NAME, "img")
                source = source_img.get_attribute("alt").strip()

                datetime = ""
                content = ""
                title = ""


                try:
                    # 기타 섹션 기사
                    datetime = driver.find_element(By.CLASS_NAME, "t11").text.strip()
                    content = ArticleParser.clear_content(driver.find_element(By.CLASS_NAME, "_article_body_contents").text.strip().replace("\n", " "))
                    title = ArticleParser.clear_headline(driver.find_element(By.CLASS_NAME, "tts_head").text.strip())

                except:
                    try:
                        # 연예 섹션 기사
                        datetime = driver.find_element(By.CLASS_NAME, "author")
                        datetime = datetime.find_element(By.TAG_NAME, "em").text.strip()
                        content = ArticleParser.clear_content(driver.find_element(By.ID, "articeBody").text.strip().replace("\n", " "))
                        title = ArticleParser.clear_headline(   driver.find_element(By.CLASS_NAME, "end_tit").text.strip()   )

                    except:
                        # 일반 섹션 기사
                        datetime = driver.find_element(By.CLASS_NAME, "info").find_element(By.TAG_NAME, "span").text.replace("기사입력", "").strip()
                        content = ArticleParser.clear_content(driver.find_element(By.CLASS_NAME, "news_end").text.strip().replace("\n", " "))
                        title = ArticleParser.clear_headline(driver.find_element(By.CLASS_NAME, "title").text.strip())



#                 print(source + "\t" + datetime  + "\t" + article_url + "\t" +  title + "\t" + content)
                print(" "*50, end="\r")


                fa.write(source + "\t" + datetime + "\t" + article_url + "\t" + title + "\t" + content + "\n")
                print()
    #--------------------------------------------------------------------------------------------------------------------------------#


            except:
                driver.close()
                time.sleep(1)
                driver.switch_to.window(current_window)
                fa.flush()
                continue


            driver.close()
            time.sleep(1)
            driver.switch_to.window(current_window)
            fa.flush()


    fa.close()


    print()
    print()
    print(int(page/10)+1, "페이지", news_count ,"개 기사 수집이 완료되었습니다.")
    print("* 수집된 기사는 아래 파일에 저장되었습니다.")
    print("  - 기사본문 :", article_filename)

                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  


50 페이지 500 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  - 기사본문 : article_주식_2021.09.02_2021.09.02_1.txt
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                          

201 번째 기사부터 최대 10개 수집중...

21 페이지 191 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  - 기사본문 : article_주식_2021.09.11_2021.09.11_1.txt
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
 

                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
251 번째 기사부터 최대 10개 수집중...

26 페이지 242 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  - 기사본문 : article_주식_2021.09.21_2021.09.21_1.txt
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
 

# 크롤링된 파일 열어서 확인하기

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./article_주식_2021.08.01_2021.10.31_1.txt', sep='\t', encoding='utf-8', header=None)

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df.columns=['신문사','날짜', 'URL','제목','본문']

In [ ]:
df.sort_values('날짜', ascending=False)